In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import model.train_schedule
import model.model_defination
import torch


In [ ]:
path = r'D:\Programming\CME_data\CME\Halo\20130830_034712_lasc2rdf_aia193rdf.png'
img = Image.open(path).convert('L')
img = np.array(img, dtype=np.float32)
print('图片大小：',img.shape)
print(img)
plt.imshow(img)
img = np.expand_dims(img, 0)

##### 预处理步骤
首先，0.5级的LASCO C2数据文件会处理为1级数据。所有的1024*1024分辨率大小的图片会被下采样至512*512。然后，会经过噪声滤波以压制部分尖锐噪声。文中使用3*3正则化正方滤波器.该滤波器是基本的线性滤波器，计算邻近像素的平均值，然后再生成差分图。
##### 图像分类
所有的(1024,1024)差分图会被下采样(112,112)大小，作为神经网络的输入。经过卷积层1，输出为(20,108,108)，经过池化层1，输出为(20,54,54)，经过卷积层2，输出为(50,50,50)，经过池化层2，输出为(50,25,25).然后经过两全连接层，得到最终的输出。

In [ ]:
def ho(h, f, padding, stride):
    return (h - f + 2 * padding) / stride + 1


In [ ]:
# 原文中:input N*112*112
print('原文中：')
print(ho(112, 5, 0, 1)) # 卷积后N*108*108
print(ho(108, 2, 0, 2)) # 池化后N*54*54
print(ho(54, 5, 0, 1)) # 卷积后N*50*50
print(ho(50, 2, 0, 2)) # 池化后N*25*25

#我的工作中采用input为 N*224*224
print('我的工作：')
print(ho(224, 5, 0, 1))  # 卷积后N*220*220
print(ho(220, 2, 0, 2))  # 池化后N*110*110
print(ho(110, 5, 0, 1))  # 卷积后N*106*106
print(ho(106, 2, 0, 2))  # 池化后N*53*53


##### CME探测
首先利用register_forward_hook函数得到卷积层输出的activation

In [ ]:
img_path = r'D:\Programming\CME_data\CME\Halo\20130830_031208_lasc2rdf_aia193rdf.png'
net = model.model_defination.Net()
parameter_path = r'D:\Programming\codetest\CMEclassfication\train_info\2022_03_27_16_54_16\parameters.pkl'
net.load_state_dict(torch.load(parameter_path))
net.eval()
activation = []


def forward_hook(module, data_input, data_output):
    activation.append(data_output)


net.conv5.register_forward_hook(forward_hook)


def infer(net, path):
    img = Image.open(path).convert('L')
    img = np.array(img, dtype=np.float32)
    img = np.expand_dims(img, 0)
    img = np.expand_dims(img, 0)
    img = torch.from_numpy(img)
    y = torch.argmax(net(img))
    return y


print(infer(net, img_path))
print(activation[0])
print(activation[0].shape)


###### CME Co-localiaztion
从卷积层的输出特征图中提取信息。利用DDT手段进行图像共定位(Co-locolization)。共定位就是在一系列的图像中找到相关物体的位置。对于一张H*W大小的图片，其activation(卷积层的输出)就是一个形状为h*w*d的三维张量。该张量可被认为有h*w个cell，每个cell包含一个d维的DD(deep desciptor)向量。  
首先，有N张图片构成的序列，这N个图片包含着同一类别的目标。这N张图片生成N个activation，每一个activation都是k*w*d维的张量。计算这N个activation的平均张量。

In [ ]:
#获得所有descriptor的平均向量
def getMeanVector(x):  #x为N*h*w*d维的np.array
    return np.mean(x, axis=(0, 1, 2))


In [ ]:
a = np.arange(12)
a = np.reshape(a, (2, 2, 3))
print(np.mean(a, axis=(0, 1)))
print(a[1][1].shape)
print('直接相乘的结果：')
print(np.matmul(a[1][1], a[1][1]))
print('升维后：')
b = np.expand_dims(a[1][1], 1)
print(b.shape)
np.matmul(b, b.T)


然后获得协方差矩阵。

In [ ]:
def cov(x: np.ndarray):  #x为N*h*w*d维的np.array
    k = x.shape[0] * x.shape[1] * x.shape[2]
    xMeanVector = getMeanVector(x)
    convMat = np.zeros(x.shape[3])
    for n in range(x.shape[0]):
        for i in range(x.shape[1]):
            for j in range(x.shape[2]):
                deviaVector = x[n][i][j] - xMeanVector
                assert deviaVector.shape == (x.shape[3], )
                # 对x中取出的descripter向量进行升维
                # 因为直接取出的descripter向量是一维的，直接相乘会出现问题，需转化为列向量
                deviaVector = np.expand_dims(deviaVector, axis=1)
                tempMat = np.matmul(deviaVector, deviaVector.T)
                convMat = convMat + tempMat
    return convMat / k
